In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import cv2
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
from keras.preprocessing import image                  
from keras.layers import Dropout, Flatten, Dense


## Loading Data

In [ ]:
BASE_DIR=('../input/state-farm-distracted-driver-detection')
train_dir=os.path.join(BASE_DIR,'imgs/train/')
test_dir=os.path.join(BASE_DIR,'imgs/test/')
df = pd.read_csv(os.path.join(BASE_DIR, 'driver_imgs_list.csv'))

In [ ]:
df.head()

In [ ]:
class_count = df.classname.value_counts()
fig = class_count.plot(kind='bar',figsize=(20,15))

In [ ]:
# Groupby subjects
by_drivers = df.groupby('subject') 
# Groupby unique drivers
unique_drivers = by_drivers.groups.keys() # drivers id
print('There are : ',len(unique_drivers), ' unique drivers')
print('There is a mean of ',round(dataset.groupby('subject').count()['classname'].mean()), ' images by driver.')

In [ ]:
NUMBER_CLASSES = 10 # 10 classes

In [ ]:
def create_csv(DATA_DIR,filename):
    class_names = os.listdir(DATA_DIR)
    data = list()
    if(os.path.isdir(os.path.join(DATA_DIR,class_names[0]))):
        for class_name in class_names:
            file_names = os.listdir(os.path.join(DATA_DIR,class_name))
            for file in file_names:
                data.append({
                    "Filename":os.path.join(DATA_DIR,class_name,file),
                    "ClassName":class_name
                })
    else:
        class_name = "test"
        file_names = os.listdir(DATA_DIR)
        for file in file_names:
            data.append(({
                "FileName":os.path.join(DATA_DIR,file),
                "ClassName":class_name
            }))
    data = pd.DataFrame(data)
    data.to_csv(os.path.join(os.getcwd(),"csv_files",filename),index=False)
CSV_FILES_DIR = os.path.join(os.getcwd(),"csv_files")
if not os.path.exists(CSV_FILES_DIR):
    os.makedirs(CSV_FILES_DIR)
    
create_csv(train_dir,"train.csv")
create_csv(test_dir,"test.csv")
train_data = pd.read_csv(os.path.join(os.getcwd(),"csv_files","train.csv"))
test_data = pd.read_csv(os.path.join(os.getcwd(),"csv_files","test.csv"))


In [ ]:
train_data.head()

In [ ]:
train_data['ClassName'].value_counts()

In [ ]:
train_data['Filename'][11]

In [ ]:
df.classname.value_counts()

### show random image

In [ ]:
num = random.randint(0, 22423)
# print(num)
class_name=df["classname"][num]
img=df["img"][num]
path=train_dir+class_name+"/"+img
print(path)
img = cv2.imread(path) 
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
print(img.shape)
plt.imshow(img)

In [ ]:
len(df)

In [ ]:
labels_list = list(set(train_data['ClassName'].values.tolist()))
labels_id = {label_name:id for id,label_name in enumerate(labels_list)}
print(labels_id)
train_data['ClassName'].replace(labels_id,inplace=True)

In [ ]:
from tensorflow.keras.utils import to_categorical
labels = to_categorical(train_data['ClassName'])
print(labels.shape)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(train_data.iloc[:,0],labels,test_size = 0.2,random_state=42,shuffle=True)

In [ ]:
x_train.shape

In [ ]:
train_data["Filename"].shape

In [ ]:
labels

In [ ]:
# train_images = train_images.reshape((22424, 28 * 28))
# train_images = train_images.astype('float32') / 255

# # test_images = test_images.reshape((10000, 28 * 28))
# # test_images = test_images.astype('float32') / 255

In [ ]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(64, 64))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in img_paths]
    return np.vstack(list_of_tensors)

In [ ]:
train_tensors = paths_to_tensor(x_train).astype('float32')/255 - 0.5

In [ ]:
valid_tensors = paths_to_tensor(x_val).astype('float32')/255 - 0.5

In [ ]:
train_tensors

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', name='Layer_1', input_shape=(64,64,3)))
network.add(layers.Dense(512, activation='relu', name='Layer_2'))
network.add(layers.Dense(256, activation='relu', name='Layer_3'))
network.add(Flatten())
network.add(layers.Dense(50, activation='relu', name='Layer_4'))
network.add(layers.Dense(10, activation='softmax'))

In [ ]:
network.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(network)

In [ ]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
network.fit(train_tensors, y_train, epochs=5, batch_size=128)

In [ ]:
test_loss, test_acc = network.evaluate(valid_tensors, y_val)